In [3]:
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os

In [5]:
# Initialize WebDriver
options = Options()
options.headless = True  # Optional: Run headless (without UI)
driver = webdriver.Chrome(options=options)
print("driver initialized")

# Open the page
driver.get('https://www.pesobility.com/stock')
time.sleep(10)
print("page loaded")

driver initialized
page loaded


In [126]:
# Get headers
heads=driver.find_elements(By.XPATH, '//thead//th')
headers=[]
for i in range(0, len(heads)):
    headers.append(heads[i].text)
headers_f=["_"+x.replace(" ", "_").replace("(%)", "").replace("-", "_").lower().strip("_") for x in headers]
print("completed header extraction. headers:")
print(headers_f)

completed header extraction. headers:
['_symbol', '_name', '_current_price', '_previous_close', '_52_week_high', '_52_week_low', '_pe', '_2023_cash_div']


In [ ]:
# get data
tables = driver.find_elements(By.XPATH, '//tbody')
data=tables[3].find_elements(By.TAG_NAME, "td") # table is the 3rd table 
print("data loaded, there are a total of:{} data points".format(len(rows)))

data loaded, there are a total of:2880 data points


In [80]:
# extract cell values from table element
data_values=list()
for i in range(0, len(data)):
    val=data[i].text
    data_values.append(val)
data_values.head()

['8990B',
 '',
 '100.9 (0.00%)',
 '100.9',
 '101 (0.10%)',
 '90.1',
 '',
 '',
 '8990P',
 '8990 Holdings - Series A Perpetual Pref',
 '101.1 (0.00%)',
 '101.1',
 '101.9 (0.79%)',
 '95.05',
 '',
 '',
 'AAA',
 'Asia Amalgamated Holdings Corp.',
 '1.61 (0.00%)',
 '1.61',
 '2.25 (39.75%)',
 '1.21',
 '-35.698',
 '',
 'AB',
 'Atok-Big Wedge Company, Inc.',
 '4.71 (-0.84%)',
 '4.75',
 '6 (27.39%)',
 '2.51',
 '747.619',
 '',
 'ABA',
 'AbaCore Capital Holdings, Inc.',
 '0.53 (-3.64%)',
 '0.55',
 '1.4 (164.15%)',
 '0.5',
 '2.386',
 '',
 'ABG',
 'Asiabest Group International, Inc.',
 '26.2 (0.00%)',
 '26.2',
 '27.55 (5.15%)',
 '2.66',
 '5,240.00',
 '',
 'ABS',
 'ABS-CBN Corporation',
 '4.2 (0.00%)',
 '4.2',
 '8.15 (94.05%)',
 '3.02',
 '1.641',
 '',
 'ABSP',
 "ABS-CBN Holdings Corporation ''PDR''",
 '3.7 (2.21%)',
 '3.62',
 '8 (116.22%)',
 '2.7',
 '',
 '',
 'AC',
 'Ayala Corporation',
 '594 (-1.66%)',
 '604',
 '740 (24.58%)',
 '562',
 '10.976',
 '11.42 (1.92%)',
 'ACE',
 'Acesite (Phils.) Hotel Cor

In [142]:
# create a table based on len(headers)
stock_prices = pd.DataFrame([data_values[i:i+len(headers)] for i in range(0, len(data_values), len(headers))], columns=[headers_f])

# format data
stock_prices._current_price=stock_prices._current_price.apply(lambda x:x.str.split(" ").str[0].astype(float))
stock_prices._previous_close=stock_prices._previous_close.apply(lambda x:x.str.split(" ").str[0].astype(float))
stock_prices._52_week_high=stock_prices._52_week_high.apply(lambda x:x.str.split(" ").str[0].astype(float))
stock_prices._52_week_low=stock_prices._52_week_low.apply(lambda x:x.str.split(" ").str[0].astype(float))

# add load datetime
stock_prices["load_datetime"]="{}-{:02d}-{:02d} {:02d}:{:02d}:{:02d}".format(
    time.localtime().tm_year
    , time.localtime().tm_mon
    , time.localtime().tm_mday
    , time.localtime().tm_hour
    , time.localtime().tm_min
    , time.localtime().tm_sec
)

stock_prices.head()


,_symbol,_name,_current_price,_previous_close,_52_week_high,_52_week_low,_pe,_2023_cash_div,load_datetime
0,8990B,,100.90,100.90,101.00,90.10,,,2024-12-19 22:23:44
1,8990P,8990 Holdings - Series A Perpetual Pref,101.10,101.10,101.90,95.05,,,2024-12-19 22:23:44
2,AAA,Asia Amalgamated Holdings Corp.,1.61,1.61,2.25,1.21,-35.698,,2024-12-19 22:23:44
3,AB,"Atok-Big Wedge Company, Inc.",4.71,4.75,6.00,2.51,747.619,,2024-12-19 22:23:44
4,ABA,"AbaCore Capital Holdings, Inc.",0.53,0.55,1.40,0.50,2.386,,2024-12-19 22:23:44


In [143]:
# save to outpath
if not os.path.isdir(os.getcwd()+"\\outputs"):
    os.mkdir(os.getcwd()+"\\outputs")
    print("{} path created".format(os.getcwd()+"\\outputs"))
outpath=os.getcwd()+"\\outputs\\stock_prices_historical.csv"
stock_prices.to_csv(outpath,mode='a')

In [94]:
driver.quit()


'2024-12-19 22:08:01'